<a href="https://colab.research.google.com/github/dntlr4463/coding/blob/main/%EB%85%BC%EB%AC%B8(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import pymysql

# DataSet 관련
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

# Visualization 관련
import matplotlib.pyplot as plt

# GPU
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available()
                     else "cpu")

# 7일간의 데이터가 입력으로 들어가고 batch size는 임의로 지정
seq_length = 7
batch = 100

# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
df_b = df_b[::-1]  
train_size = int(len(df_b)*0.7)
train_set = df_b[0:train_size]  
test_set = df_b[train_size-seq_length:]

# Input scale
scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:, :-1])

train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])

# Output scale
scaler_y = MinMaxScaler()
scaler_y.fit(train_set.iloc[:, [-1]])

train_set.iloc[:, -1] = scaler_y.transform(train_set.iloc[:, [-1]])
test_set.iloc[:, -1] = scaler_y.transform(test_set.iloc[:, [-1]])

In [ ]:
# 데이터셋 생성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series)-seq_length):
        _x = time_series[i:i+seq_length, :]
        _y = time_series[i+seq_length, [-1]]
        # print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(np.array(train_set), seq_length)
testX, testY = build_dataset(np.array(test_set), seq_length)

# 텐서로 변환
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

# 텐서 형태로 데이터 정의
dataset = TensorDataset(trainX_tensor, trainY_tensor)

# 데이터로더는 기본적으로 2개의 인자를 입력받으며 배치크기는 통상적으로 2의 배수를 사용
dataloader = DataLoader(dataset,
                        batch_size=batch,
                        shuffle=True,  
                        drop_last=True)

In [ ]:
# hyperparameter
data_dim = 5
hidden_dim = 10
output_dim = 1 
learning_rate = 0.01
nb_epochs = 100

class Net(nn.Module):
    # # 기본변수, layer를 초기화해주는 생성자
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers,
                            # dropout = 0.1,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True) 
        
    # 은닉층을 초기화하는 함수
    # 매번 새로운 학습을 하여 최적의 값을 찾기 위한 방안인듯.
    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))
    
    # 순전파 함수
    def forward(self, x):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

In [ ]:
def train_model(model, train_df, epochs=None, lr=None, verbose=10, patience=10):
    # 손실함수
    criterion = nn.MSELoss().to(device)
    # 최적함수
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # epoch마다 loss 저장
    train_hist = np.zeros(epochs)
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        for batch_idx, samples in enumerate(train_df):
            x_train, y_train = samples
            # seq별 hidden state reset
            model.reset_hidden_state()
            
            # H(X) 계산
            outputs = model(x_train)
            # cost 계산
            loss = criterion(outputs, y_train)
            # cost로 H(X) 개선
            
            # pytorch에서는 gradients 값들을 backward 해줄 때 계속 더해주기 때문에 backpropagation 하기 전에 gradients를 zero로 만들어주고 시작
            # 즉, 한 번의 학습이 완료되면 gradients를 항상 0으로 만들어주어야 하고 그렇지 않으면 gradient 가 의도한 방향과 다른 방향을 가리켜 학습이 잘 이루어지지 않는다.
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient Clipping 적용
            torch_utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            avg_cost += loss.item()
            #avg_cost += loss/total_batch
            
        train_hist[epoch] = avg_cost
        
        if epoch%verbose == 0 :
            print('Epoch : ', '%04d' % (epoch), 'train loss : ', '{:.4f}'.format(avg_cost))
            
        # patience마다 early stopping 여부 확인
        if (epoch%patience==0) and (epoch!=0):
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping')
                break
    return model.eval(), train_hist

In [ ]:
LSTM = Net(data_dim, hidden_dim, seq_length, output_dim, 1).to(device)

# Train method
model, train_hist = train_model(LSTM, dataloader, epochs = epochs, lr = learning_rate, verbose = 20, patience=10)